In [1]:
import numpy as np
import pandas as pd
pd.options.display.min_rows = 63

In [5]:
def season_cleaner(team_df):
    season_list = [1995, 1996, 1997, 1998, 1999,
                   2000, 2001, 2002, 2003, 2004, 
                   2005, 2006, 2007, 2008, 2009,
                   2010, 2011, 2012, 2013, 2014,
                   2015, 2016, 2017, 2018, 2019]
    team_seasons = []
    for season in season_list:
        #converts date column to datetimeobject
        team_df['date'] = pd.to_datetime(team_df['date'])
        df = team_df[team_df['date'].dt.year == season].copy()
        #datetime setup
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day
        df['weekday'] = df['date'].dt.weekday
        df['weekday_name'] = df['date'].dt.weekday_name
        wins = []
        losses = []
        for wl in df['record']:
            w, l = wl.split('-')
            wins.append(int(w))
            losses.append(int(l))
        df['wins'] = wins
        df['losses'] = losses
        df['total_games'] = df['wins'] + df['losses']
        df['win_pct'] = df['wins'] / df['total_games']
        #drop attendance nan value rows
        df.dropna(subset=['attendance'], inplace=True)
        #attendance convert from string to int
        df['attendance'] = df['attendance'].str.replace(',', '').astype(int)
        ### streak convert ####
        df['streak'].fillna('0', inplace=True)
        df['streak'] = [len(x) if '+' in x else -len(x) if '-' in x else 0 for x in df['streak']]
        #games back convert
        df['games_back'] = [0. if x == ' Tied' or x == '0'else float(x.replace('up', '')) if 'up' in x else -float(x) for x in df['games_back']]
        # average runs per game feature
        df['runs_pg_avg'] = [0 if x == min(df['date']) else df[df['date'] < x].r.astype(int).mean() for x in df['date']]
        #5 game moving runs average 
        df['runs_mavg_5'] = df['r'].rolling(5).mean().shift()
        # 5 game moving runs against average
        df['runs_against_mavg_5'] = df['ra'].rolling(5).mean().shift()
        #ten game moving win percentage
        ten_game_win_perc = [list(df['win_loss'][i-10:i]).count('W') / 10. for i in range(10, len(df))]
        for i in range(10):
            ten_game_win_perc.insert(0, None)
        df['ten_game_win_perc'] = ten_game_win_perc
        # 5 game era moving average
        df['era_mavg_5'] = df['pitch_era'].rolling(5).mean().shift()
        # 5 game batting average moving average
        df['bat_avg_mavg_5'] = df['bat_batting_avg'].rolling(5).mean().shift()
        # 5 game bat ops moving average
        df['ops_mavg_5'] = df['bat_ops'].rolling(5).mean().shift()
        # average hits per game
        df['hits_pg_avg'] = [0 if x == min(df['date']) else df[df['date'] < x].bat_hits.mean() for x in df['date']]
        # 5 game hits moving average
        df['hits_mavg_5'] = df['bat_hits'].rolling(5).mean().shift()
        # batting strike outs per game
        df['bat_stkout_pg_avg'] = [0 if x == min(df['date']) else df[df['date'] < x].bat_strike_outs.mean() for x in df['date']]
        # pitching strikeout per game
        df['ptich_stkout_pg_avg'] = [0 if x == min(df['date']) else df[df['date'] < x].pitch_strike_outs.mean() for x in df['date']]
        # 5 game pitching strikeout moving average
        df['pitch_strkout_mavg_5'] = df['pitch_strike_outs'].rolling(5).mean().shift()
        # opening day dummy
        df['opening_day'] = [1 if x == min(df['date']) else 0 for x in df['date']]
        #night game dummy
        df['night_game'] = [1 if x == 'N' else 0 for x in df['day_night']]
        #fill nan values created by the rolling means with values from the same column
        df['runs_mavg_5'] = [df.iloc[i].r if np.isnan(j) else j for i, j in enumerate(df['runs_mavg_5'])]
        df['runs_against_mavg_5'] = [df.iloc[i].ra if np.isnan(j) else j for i, j in enumerate(df['runs_against_mavg_5'])]
        df['ten_game_win_perc'] = [df.iloc[i].win_pct if np.isnan(j) else j for i, j in enumerate(df['ten_game_win_perc'])]
        df['era_mavg_5'] = [df.iloc[i].pitch_era if np.isnan(j) else j for i, j in enumerate(df['era_mavg_5'])]
        df['bat_avg_mavg_5'] = [df.iloc[i].bat_batting_avg if np.isnan(j) else j for i, j in enumerate(df['bat_avg_mavg_5'])]
        df['ops_mavg_5'] = [df.iloc[i].bat_ops if np.isnan(j) else j for i, j in enumerate(df['ops_mavg_5'])]
        df['hits_mavg_5'] = [df.iloc[i].bat_hits if np.isnan(j) else j for i, j in enumerate(df['hits_mavg_5'])]
        df['pitch_strkout_mavg_5'] = [df.iloc[i].pitch_strike_outs if np.isnan(j) else j for i, j in enumerate(df['pitch_strkout_mavg_5'])]
        #drop all away games from season data
        df['home_away'].fillna('home',inplace=True)
        df = df[~df['home_away'].str.contains('@')].reset_index(drop=True)

        team_seasons.append(df)
        print(f'{season} done')
    club = pd.concat(team_seasons)
    print('name for file:')
    a = input()
    club.to_csv(f'../data/clean_data/{a}_clean.csv', index=False)
        
    return print(f'all seasons for {a} done.')

In [3]:
houston = pd.read_csv('../data/pre_clean_data/HOU/houston_pc.csv')

In [4]:
%time season_cleaner(houston)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
houston
all seasons for       game#        day       date team home_away  opp win_loss     r    ra  \
0         1  Wednesday 1995-04-26  HOU         @  SDP        W  10.0   2.0   
1         2   Thursday 1995-04-27  HOU         @  SDP        L   1.0  13.0   
2         3     Friday 1995-04-28  HOU       NaN  COL        L   1.0   2.0   
3         4   Saturday 1995-04-29  HOU       NaN  COL        L   1.0   2.0   
4         5     Sunday 1995-04-30  HOU       NaN  COL        W   3.0   1.0   
5         6    Tuesday 1995-05-02  HOU         @  CHC        W   5.0   2.0   
6         7  Wednesday 1995-05-03  HOU         @  CHC        W  11.0   2.0   
7         8   Thursday 1995-05-04  HOU         @  STL        W   6.0   4.0   
8        

In [6]:
arizona = pd.read_csv('../data/pre_clean_data/ARI/arizona_pc.csv')

In [10]:
atlanta = pd.read_csv('../data/pre_clean_data/ATL/atlanta_pc.csv')

In [11]:
season_cleaner(atlanta)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
atlanta
all seasons for atlanta done.


In [12]:
baltimore = pd.read_csv('../data/pre_clean_data/BAL/baltimore_pc.csv')

In [13]:
season_cleaner(baltimore)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
baltimore
all seasons for baltimore done.


In [14]:
boston = pd.read_csv('../data/pre_clean_data/BOS/boston_pc.csv')

In [15]:
season_cleaner(boston)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
boston
all seasons for boston done.


In [16]:
cubs = pd.read_csv('../data/pre_clean_data/CHC/cubs_pc.csv')

In [17]:
season_cleaner(cubs)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
cubs
all seasons for cubs done.


In [18]:
white_sox = pd.read_csv('../data/pre_clean_data/CHW/chicago_white_sox_pc.csv')

In [19]:
season_cleaner(white_sox)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
white_sox
all seasons for white_sox done.


In [20]:
reds = pd.read_csv('../data/pre_clean_data/CIN/reds_pc.csv')

In [21]:
season_cleaner(reds)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
reds
all seasons for reds done.


In [22]:
indians = pd.read_csv('../data/pre_clean_data/CLE/cleveland_pc.csv')

In [23]:
season_cleaner(indians)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
cleveland
all seasons for cleveland done.


In [24]:
rockies = pd.read_csv('../data/pre_clean_data/COL/colorado_pc.csv')

In [25]:
season_cleaner(rockies)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
colorado
all seasons for colorado done.


In [26]:
detroit = pd.read_csv('../data/pre_clean_data/DET/detroit_pc.csv')

In [27]:
season_cleaner(detroit)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
detroit
all seasons for detroit done.


In [28]:
royals = pd.read_csv('../data/pre_clean_data/KCR/kansas_city_pc.csv')

In [29]:
season_cleaner(royals)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
kansas_city
all seasons for kansas_city done.


In [30]:
angels = pd.read_csv('../data/pre_clean_data/LAA/angels_pc.csv')

In [31]:
season_cleaner(angels)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
angels
all seasons for angels done.


In [32]:
dodgers = pd.read_csv('../data/pre_clean_data/LAD/dodgers_pc.csv')

In [33]:
season_cleaner(dodgers)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
dodgers
all seasons for dodgers done.


In [34]:
miami = pd.read_csv('../data/pre_clean_data/MIA/marlins_pc.csv')

In [35]:
season_cleaner(miami)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
miami
all seasons for miami done.


In [36]:
brewers = pd.read_csv('../data/pre_clean_data/MIL/brewers_pc.csv')

In [37]:
season_cleaner(brewers)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
brewers
all seasons for brewers done.


In [38]:
minnesota = pd.read_csv('../data/pre_clean_data/MIN/minnesota_pc.csv')

In [39]:
season_cleaner(minnesota)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
minnesota
all seasons for minnesota done.


In [40]:
mets = pd.read_csv('../data/pre_clean_data/NYM/mets_pc.csv')

In [41]:
season_cleaner(mets)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
mets
all seasons for mets done.


In [42]:
yankees = pd.read_csv('../data/pre_clean_data/NYY/new_york_pc.csv')

In [43]:
season_cleaner(yankees)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
yankees
all seasons for yankees done.


In [44]:
oakland = pd.read_csv('../data/pre_clean_data/OAK/oakland_pc.csv')

In [45]:
season_cleaner(oakland)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
oakland
all seasons for oakland done.


In [46]:
phillies = pd.read_csv('../data/pre_clean_data/PHI/philadelphia_pc.csv')

In [47]:
season_cleaner(phillies)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
phillies
all seasons for phillies done.


In [48]:
pirates = pd.read_csv('../data/pre_clean_data/PIT/pirates_pc.csv')

In [49]:
season_cleaner(pirates)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
pirates
all seasons for pirates done.


In [50]:
padres = pd.read_csv('../data/pre_clean_data/SDP/san_diego_pc.csv')

In [51]:
season_cleaner(padres)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
padres
all seasons for padres done.


In [52]:
seattle = pd.read_csv('../data/pre_clean_data/SEA/seattle_pc.csv')

In [53]:
season_cleaner(seattle)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
seattle
all seasons for seattle done.


In [54]:
giants = pd.read_csv('../data/pre_clean_data/SFG/san_francisco_pc.csv')

In [55]:
season_cleaner(giants)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
giants
all seasons for giants done.


In [56]:
cardinals = pd.read_csv('../data/pre_clean_data/STL/cardinals_pc.csv')

In [57]:
season_cleaner(cardinals)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
cardinals
all seasons for cardinals done.


In [58]:
rangers = pd.read_csv('../data/pre_clean_data/TEX/rangers_pc.csv')

In [59]:
season_cleaner(rangers)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
rangers
all seasons for rangers done.


In [60]:
toronto = pd.read_csv('../data/pre_clean_data/TOR/toronto_pc.csv')

In [61]:
season_cleaner(toronto)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
toronto
all seasons for toronto done.


In [62]:
nationals = pd.read_csv('../data/pre_clean_data/WSN/washington_pc.csv')

In [63]:
season_cleaner(nationals)

1995 done
1996 done
1997 done
1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
nationals
all seasons for nationals done.


In [64]:
def season_cleaner_2(team_df):
    season_list = [1998, 1999,
                   2000, 2001, 2002, 2003, 2004, 
                   2005, 2006, 2007, 2008, 2009,
                   2010, 2011, 2012, 2013, 2014,
                   2015, 2016, 2017, 2018, 2019]
    team_seasons = []
    for season in season_list:
        #converts date column to datetimeobject
        team_df['date'] = pd.to_datetime(team_df['date'])
        df = team_df[team_df['date'].dt.year == season].copy()
        #datetime setup
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day
        df['weekday'] = df['date'].dt.weekday
        df['weekday_name'] = df['date'].dt.weekday_name
        wins = []
        losses = []
        for wl in df['record']:
            w, l = wl.split('-')
            wins.append(int(w))
            losses.append(int(l))
        df['wins'] = wins
        df['losses'] = losses
        df['total_games'] = df['wins'] + df['losses']
        df['win_pct'] = df['wins'] / df['total_games']
        #drop attendance nan value rows
        df.dropna(subset=['attendance'], inplace=True)
        #attendance convert from string to int
        df['attendance'] = df['attendance'].str.replace(',', '').astype(int)
        ### streak convert ####
        df['streak'].fillna('0', inplace=True)
        df['streak'] = [len(x) if '+' in x else -len(x) if '-' in x else 0 for x in df['streak']]
        #games back convert
        df['games_back'] = [0. if x == ' Tied' or x == '0'else float(x.replace('up', '')) if 'up' in x else -float(x) for x in df['games_back']]
        # average runs per game feature
        df['runs_pg_avg'] = [0 if x == min(df['date']) else df[df['date'] < x].r.astype(int).mean() for x in df['date']]
        #5 game moving runs average 
        df['runs_mavg_5'] = df['r'].rolling(5).mean().shift()
        # 5 game moving runs against average
        df['runs_against_mavg_5'] = df['ra'].rolling(5).mean().shift()
        #ten game moving win percentage
        ten_game_win_perc = [list(df['win_loss'][i-10:i]).count('W') / 10. for i in range(10, len(df))]
        for i in range(10):
            ten_game_win_perc.insert(0, None)
        df['ten_game_win_perc'] = ten_game_win_perc
        # 5 game era moving average
        df['era_mavg_5'] = df['pitch_era'].rolling(5).mean().shift()
        # 5 game batting average moving average
        df['bat_avg_mavg_5'] = df['bat_batting_avg'].rolling(5).mean().shift()
        # 5 game bat ops moving average
        df['ops_mavg_5'] = df['bat_ops'].rolling(5).mean().shift()
        # average hits per game
        df['hits_pg_avg'] = [0 if x == min(df['date']) else df[df['date'] < x].bat_hits.mean() for x in df['date']]
        # 5 game hits moving average
        df['hits_mavg_5'] = df['bat_hits'].rolling(5).mean().shift()
        # batting strike outs per game
        df['bat_stkout_pg_avg'] = [0 if x == min(df['date']) else df[df['date'] < x].bat_strike_outs.mean() for x in df['date']]
        # pitching strikeout per game
        df['ptich_stkout_pg_avg'] = [0 if x == min(df['date']) else df[df['date'] < x].pitch_strike_outs.mean() for x in df['date']]
        # 5 game pitching strikeout moving average
        df['pitch_strkout_mavg_5'] = df['pitch_strike_outs'].rolling(5).mean().shift()
        # opening day dummy
        df['opening_day'] = [1 if x == min(df['date']) else 0 for x in df['date']]
        #night game dummy
        df['night_game'] = [1 if x == 'N' else 0 for x in df['day_night']]
        #fill nan values created by the rolling means with values from the same column
        df['runs_mavg_5'] = [df.iloc[i].r if np.isnan(j) else j for i, j in enumerate(df['runs_mavg_5'])]
        df['runs_against_mavg_5'] = [df.iloc[i].ra if np.isnan(j) else j for i, j in enumerate(df['runs_against_mavg_5'])]
        df['ten_game_win_perc'] = [df.iloc[i].win_pct if np.isnan(j) else j for i, j in enumerate(df['ten_game_win_perc'])]
        df['era_mavg_5'] = [df.iloc[i].pitch_era if np.isnan(j) else j for i, j in enumerate(df['era_mavg_5'])]
        df['bat_avg_mavg_5'] = [df.iloc[i].bat_batting_avg if np.isnan(j) else j for i, j in enumerate(df['bat_avg_mavg_5'])]
        df['ops_mavg_5'] = [df.iloc[i].bat_ops if np.isnan(j) else j for i, j in enumerate(df['ops_mavg_5'])]
        df['hits_mavg_5'] = [df.iloc[i].bat_hits if np.isnan(j) else j for i, j in enumerate(df['hits_mavg_5'])]
        df['pitch_strkout_mavg_5'] = [df.iloc[i].pitch_strike_outs if np.isnan(j) else j for i, j in enumerate(df['pitch_strkout_mavg_5'])]
        #drop all away games from season data
        df['home_away'].fillna('home',inplace=True)
        df = df[~df['home_away'].str.contains('@')].reset_index(drop=True)

        team_seasons.append(df)
        print(f'{season} done')
    club = pd.concat(team_seasons)
    print('name for file:')
    a = input()
    club.to_csv(f'../data/clean_data/{a}_clean.csv', index=False)
        
    return print(f'all seasons for {a} done.')

In [65]:
season_cleaner_2(arizona)

1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
arizona
all seasons for arizona done.


In [66]:
rays = pd.read_csv('../data/pre_clean_data/TBR/tampa_bay_pc.csv')

In [67]:
season_cleaner_2(rays)

1998 done
1999 done
2000 done
2001 done
2002 done
2003 done
2004 done
2005 done
2006 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
name for file:
tampa_bay
all seasons for tampa_bay done.


In [68]:
HOU = pd.read_csv('../data/clean_data/houston_clean.csv')
TEX = pd.read_csv('../data/clean_data/rangers_clean.csv')
SEA = pd.read_csv('../data/clean_data/seattle_clean.csv')
OAK = pd.read_csv('../data/clean_data/oakland_clean.csv')
LAA = pd.read_csv('../data/clean_data/angels_clean.csv')
NYY = pd.read_csv('../data/clean_data/yankees_clean.csv')
BOS = pd.read_csv('../data/clean_data/boston_clean.csv')
TOR = pd.read_csv('../data/clean_data/toronto_clean.csv')
BAL = pd.read_csv('../data/clean_data/baltimore_clean.csv')
TBR = pd.read_csv('../data/clean_data/tampa_bay_clean.csv')
MIN = pd.read_csv('../data/clean_data/minnesota_clean.csv')
CLE = pd.read_csv('../data/clean_data/cleveland_clean.csv')
CHW = pd.read_csv('../data/clean_data/white_sox_clean.csv')
KCR = pd.read_csv('../data/clean_data/kansas_city_clean.csv')
DET = pd.read_csv('../data/clean_data/detroit_clean.csv')
LAD = pd.read_csv('../data/clean_data/dodgers_clean.csv')
SFG = pd.read_csv('../data/clean_data/giants_clean.csv')
COL = pd.read_csv('../data/clean_data/colorado_clean.csv')
SDP = pd.read_csv('../data/clean_data/padres_clean.csv')
ARI = pd.read_csv('../data/clean_data/arizona_clean.csv')
ATL = pd.read_csv('../data/clean_data/atlanta_clean.csv')
NYM = pd.read_csv('../data/clean_data/mets_clean.csv')
PHI = pd.read_csv('../data/clean_data/phillies_clean.csv')
WSN = pd.read_csv('../data/clean_data/nationals_clean.csv')
MIA = pd.read_csv('../data/clean_data/miami_clean.csv')
STL = pd.read_csv('../data/clean_data/cardinals_clean.csv')
MIL = pd.read_csv('../data/clean_data/brewers_clean.csv')
CHC = pd.read_csv('../data/clean_data/cubs_clean.csv')
CIN = pd.read_csv('../data/clean_data/reds_clean.csv')
PIT = pd.read_csv('../data/clean_data/pirates_clean.csv')

In [69]:
mlb_df = pd.concat([HOU, TEX, SEA, OAK, LAA, NYY, BOS, TOR, BAL, TBR, 
                    MIN, CLE, CHW, KCR, DET, LAD, SFG, COL, SDP, ARI, 
                    ATL, NYM, PHI, WSN, MIA, STL, MIL, CHC, CIN, PIT], sort = False)

In [71]:
mlb_df.to_csv('../data/clean_data/mlb_df.csv', index=False)

In [72]:
HOU = pd.read_csv('../data/stadiums/HOU_stadium_data.csv')
TEX = pd.read_csv('../data/stadiums/TEX_stadium_data.csv')
SEA = pd.read_csv('../data/stadiums/SEA_stadium_data.csv')
OAK = pd.read_csv('../data/stadiums/OAK_stadium_data.csv')
LAA = pd.read_csv('../data/stadiums/LAA_stadium_data.csv')
NYY = pd.read_csv('../data/stadiums/NYY_stadium_data.csv')
BOS = pd.read_csv('../data/stadiums/BOS_stadium_data.csv')
TOR = pd.read_csv('../data/stadiums/TOR_stadium_data.csv')
BAL = pd.read_csv('../data/stadiums/BAL_stadium_data.csv')
TBR = pd.read_csv('../data/stadiums/TBR_stadium_data.csv')
MIN = pd.read_csv('../data/stadiums/MIN_stadium_data.csv')
CLE = pd.read_csv('../data/stadiums/CLE_stadium_data.csv')
CHW = pd.read_csv('../data/stadiums/CHW_stadium_data.csv')
KCR = pd.read_csv('../data/stadiums/KCR_stadium_data.csv')
DET = pd.read_csv('../data/stadiums/DET_stadium_data.csv')
LAD = pd.read_csv('../data/stadiums/LAD_stadium_data.csv')
SFG = pd.read_csv('../data/stadiums/SFG_stadium_data.csv')
COL = pd.read_csv('../data/stadiums/COL_stadium_data.csv')
SDP = pd.read_csv('../data/stadiums/SDP_stadium_data.csv')
ARI = pd.read_csv('../data/stadiums/ARI_stadium_data.csv')
ATL = pd.read_csv('../data/stadiums/ATL_stadium_data.csv')
NYM = pd.read_csv('../data/stadiums/NYM_stadium_data.csv')
PHI = pd.read_csv('../data/stadiums/PHI_stadium_data.csv')
WSN = pd.read_csv('../data/stadiums/WSN_stadium_data.csv')
MIA = pd.read_csv('../data/stadiums/MIA_stadium_data.csv')
STL = pd.read_csv('../data/stadiums/STL_stadium_data.csv')
MIL = pd.read_csv('../data/stadiums/MIL_stadium_data.csv')
CHC = pd.read_csv('../data/stadiums/CHC_stadium_data.csv')
CIN = pd.read_csv('../data/stadiums/CIN_stadium_data.csv')
PIT = pd.read_csv('../data/stadiums/Pit_stadium_data.csv')

In [73]:
mlb_stadiums = pd.concat([HOU, TEX, SEA, OAK, LAA, NYY, BOS, TOR, BAL, TBR, 
                    MIN, CLE, CHW, KCR, DET, LAD, SFG, COL, SDP, ARI, 
                    ATL, NYM, PHI, WSN, MIA, STL, MIL, CHC, CIN, PIT], sort = False)

In [75]:
mlb_stadiums.to_csv('../data/stadiums/mlb_stadiums.csv', index=False)

team        object
year         int64
state       object
stadium     object
capacity     int64
dtype: object

In [77]:
HOU = pd.read_csv('../data/franchise/HOU_franchise.csv')
TEX = pd.read_csv('../data/franchise/TEX_franchise.csv')
SEA = pd.read_csv('../data/franchise/SEA_franchise.csv')
OAK = pd.read_csv('../data/franchise/OAK_franchise.csv')
LAA = pd.read_csv('../data/franchise/ANA_franchise.csv')
NYY = pd.read_csv('../data/franchise/NYY_franchise.csv')
BOS = pd.read_csv('../data/franchise/BOS_franchise.csv')
TOR = pd.read_csv('../data/franchise/TOR_franchise.csv')
BAL = pd.read_csv('../data/franchise/BAL_franchise.csv')
TBR = pd.read_csv('../data/franchise/TBD_franchise.csv')
MIN = pd.read_csv('../data/franchise/MIN_franchise.csv')
CLE = pd.read_csv('../data/franchise/CLE_franchise.csv')
CHW = pd.read_csv('../data/franchise/CHW_franchise.csv')
KCR = pd.read_csv('../data/franchise/KCR_franchise.csv')
DET = pd.read_csv('../data/franchise/DET_franchise.csv')
LAD = pd.read_csv('../data/franchise/LAD_franchise.csv')
SFG = pd.read_csv('../data/franchise/SFG_franchise.csv')
COL = pd.read_csv('../data/franchise/COL_franchise.csv')
SDP = pd.read_csv('../data/franchise/SDP_franchise.csv')
ARI = pd.read_csv('../data/franchise/ARI_franchise.csv')
ATL = pd.read_csv('../data/franchise/ATL_franchise.csv')
NYM = pd.read_csv('../data/franchise/NYM_franchise.csv')
PHI = pd.read_csv('../data/franchise/PHI_franchise.csv')
WSN = pd.read_csv('../data/franchise/WSN_franchise.csv')
MIA = pd.read_csv('../data/franchise/FLA_franchise.csv')
STL = pd.read_csv('../data/franchise/STL_franchise.csv')
MIL = pd.read_csv('../data/franchise/MIL_franchise.csv')
CHC = pd.read_csv('../data/franchise/CHC_franchise.csv')
CIN = pd.read_csv('../data/franchise/CIN_franchise.csv')
PIT = pd.read_csv('../data/franchise/PIT_franchise.csv')

In [78]:
mlb_franchise = pd.concat([HOU, TEX, SEA, OAK, LAA, NYY, BOS, TOR, BAL, TBR, 
                    MIN, CLE, CHW, KCR, DET, LAD, SFG, COL, SDP, ARI, 
                    ATL, NYM, PHI, WSN, MIA, STL, MIL, CHC, CIN, PIT], sort = False)

In [83]:
mlb_franchise['season_total_attendance'] = mlb_franchise['season_total_attendance'].str.replace(',', '').astype(int)



In [85]:
mlb_franchise.to_csv('../data/franchise/mlb_franchise.csv', index=False)

In [86]:
HOU = pd.read_csv('../data/attendance/HOU_attendance.csv')
TEX = pd.read_csv('../data/attendance/TEX_attendance.csv')
SEA = pd.read_csv('../data/attendance/SEA_attendance.csv')
OAK = pd.read_csv('../data/attendance/OAK_attendance.csv')
LAA = pd.read_csv('../data/attendance/ANA_attendance.csv')
NYY = pd.read_csv('../data/attendance/NYY_attendance.csv')
BOS = pd.read_csv('../data/attendance/BOS_attendance.csv')
TOR = pd.read_csv('../data/attendance/TOR_attendance.csv')
BAL = pd.read_csv('../data/attendance/BAL_attendance.csv')
TBR = pd.read_csv('../data/attendance/TBD_attendance.csv')
MIN = pd.read_csv('../data/attendance/MIN_attendance.csv')
CLE = pd.read_csv('../data/attendance/CLE_attendance.csv')
CHW = pd.read_csv('../data/attendance/CHW_attendance.csv')
KCR = pd.read_csv('../data/attendance/KCR_attendance.csv')
DET = pd.read_csv('../data/attendance/DET_attendance.csv')
LAD = pd.read_csv('../data/attendance/LAD_attendance.csv')
SFG = pd.read_csv('../data/attendance/SFG_attendance.csv')
COL = pd.read_csv('../data/attendance/COL_attendance.csv')
SDP = pd.read_csv('../data/attendance/SDP_attendance.csv')
ARI = pd.read_csv('../data/attendance/ARI_attendance.csv')
ATL = pd.read_csv('../data/attendance/ATL_attendance.csv')
NYM = pd.read_csv('../data/attendance/NYM_attendance.csv')
PHI = pd.read_csv('../data/attendance/PHI_attendance.csv')
WSN = pd.read_csv('../data/attendance/WSN_attendance.csv')
MIA = pd.read_csv('../data/attendance/FLA_attendance.csv')
STL = pd.read_csv('../data/attendance/STL_attendance.csv')
MIL = pd.read_csv('../data/attendance/MIL_attendance.csv')
CHC = pd.read_csv('../data/attendance/CHC_attendance.csv')
CIN = pd.read_csv('../data/attendance/CIN_attendance.csv')
PIT = pd.read_csv('../data/attendance/PIT_attendance.csv')

In [87]:
mlb_attendance = pd.concat([HOU, TEX, SEA, OAK, LAA, NYY, BOS, TOR, BAL, TBR, 
                    MIN, CLE, CHW, KCR, DET, LAD, SFG, COL, SDP, ARI, 
                    ATL, NYM, PHI, WSN, MIA, STL, MIL, CHC, CIN, PIT], sort = False)

In [91]:
mlb_attendance['est_payroll'] = mlb_attendance['est_payroll'].replace('[\$,]', '', regex=True).astype(float)

In [94]:
mlb_attendance['attendance_game_avg'] = mlb_attendance['attendance_game_avg'].str.replace(',', '').astype(int)

In [97]:
mlb_attendance.to_csv('../data/attendance/mlb_attendance.csv', index=False)

In [100]:
mlb_stadiums.dtypes

team        object
year         int64
state       object
stadium     object
capacity     int64
dtype: object

In [101]:
mlb_df = mlb_df.merge(mlb_stadiums.drop('state', axis = 1), on =['team', 'year'])

In [103]:
mlb

,game#,day,date,team,home_away,opp,win_loss,r,ra,innings,...,ops_mavg_5,hits_pg_avg,hits_mavg_5,bat_stkout_pg_avg,ptich_stkout_pg_avg,pitch_strkout_mavg_5,opening_day,night_game,stadium,capacity
0,3,28,1995-04-28,HOU,home,COL,L,1.0,2.0,NaN,...,0.6830,8.500000,7.0,9.500000,8.000000,6.0,0,1,Astrodome,54370
1,4,29,1995-04-29,HOU,home,COL,L,1.0,2.0,NaN,...,0.4620,8.000000,4.0,9.000000,7.333333,8.0,0,1,Astrodome,54370
2,5,30,1995-04-30,HOU,home,COL,W,3.0,1.0,NaN,...,0.5130,7.000000,5.0,8.750000,7.500000,12.0,0,0,Astrodome,54370
3,15,12,1995-05-12,HOU,home,PHI,L,2.0,5.0,NaN,...,0.8970,9.928571,12.4,6.642857,7.000000,6.2,0,1,Astrodome,54370
4,16,13,1995-05-13,HOU,home,PHI,L,5.0,7.0,NaN,...,0.8718,9.800000,12.6,6.733333,6.866667,6.2,0,1,Astrodome,54370
5,17,14,1995-05-14,HOU,home,PHI,L,2.0,5.0,NaN,...,0.8004,9.625000,11.2,6.625000,6.562500,5.2,0,0,Astrodome,54370
6,18,16,1995-05-16,HOU,home,NYM,L,0.0,1.0,NaN,...,0.7642,9.352941,10.6,6.588235,6.529412,4.8,0,1,Astrodome,54370
7,19,17,1995-05-17,HOU,home,NYM,W,7.0,2.0,NaN,...,0.6474,9.000000,8.0,6.611111,6.444444,4.2,0,1,Astrodome,54370
8,20,18,1995-05-18,HOU,home,NYM,L,1.0,8.0,NaN,...,0.6234,9.263158,7.4,6.631579,6.315789,4.4,0,1,Astrodome,54370
9,21,19,1995-05-19,HOU,home,MON,W,10.0,2.0,NaN,...,0.5882,9.100000,7.0,6.500000,6.400000,5.0,0,1,Astrodome,54370


In [104]:
mlb_franchise

,year,team_name,league,total_games,wins,losses,ties,season_win_loss_%,pythagorean_win_loss_%,division_finish,division_game_back,runs_for,runs_against,season_total_attendance,average_batter_age,average_pitcher_age,number_of_batters,number_of_pitchers,top_player,manager
0,2019,Houston Astros,AL West,162,107,55,0,0.660,0.660,1st of 5,--,920,640,2857367,28.9,29.9,45,26,A.Bregman (8.4),A.Hinch (107-55)
1,2018,Houston Astros,AL West,162,103,59,0,0.636,0.675,1st of 5,--,797,534,2980549,28.1,30.0,41,22,A.Bregman (6.9),A.Hinch (103-59)
2,2017,Houston Astros,AL West,162,101,61,0,0.623,0.611,1st of 5,--,896,700,2403671,28.8,28.5,46,27,J.Altuve (8.1),A.Hinch (101-61)
3,2016,Houston Astros,AL West,162,84,78,0,0.519,0.515,3rd of 5,11.0,724,701,2306623,26.4,28.9,43,23,J.Altuve (7.7),A.Hinch (84-78)
4,2015,Houston Astros,AL West,162,86,76,0,0.531,0.575,2nd of 5,2.0,729,618,2153585,26.3,29.4,46,24,D.Keuchel (6.7),A.Hinch (86-76)
5,2014,Houston Astros,AL West,162,70,92,0,0.432,0.437,4th of 5,28.0,629,723,1751829,25.4,27.9,48,28,J.Altuve (6.1),B.Porter (59-79) and T.Lawless (11-13)
6,2013,Houston Astros,AL West,162,51,111,0,0.315,0.354,5th of 5,45.0,610,848,1651883,25.9,27.2,50,26,J.Castro (4.1),B.Porter (51-111)
7,2012,Houston Astros,NL Central,162,55,107,0,0.340,0.362,6th of 6,42.0,583,794,1607733,26.6,27.2,50,26,L.Harrell (3.1),B.Mills (39-82) and T.DeFrancesco (16-25)
8,2011,Houston Astros,NL Central,162,56,106,0,0.346,0.384,6th of 6,40.0,615,796,2067016,28.2,27.2,47,23,C.Lee (3.9),B.Mills (56-106)
9,2010,Houston Astros,NL Central,162,76,86,0,0.469,0.420,4th of 6,15.0,611,729,2331490,29.5,29.6,48,25,M.Bourn (5.5),B.Mills (76-86)


In [106]:
#playoff dummy for 0 whem missed playoffs, 1 when made playoffs, 2 when won world series
mlb_attendance['playoffs'] = [0 if str(i) == 'nan' else 2 if 'Won' in str(i) else 1 for i in mlb_attendance['playoffs']]

In [108]:
mlb_attendance.to_csv('../data/attendance/mlb_attendance.csv', index=False)

In [110]:
mlb_franchise = pd.read_csv('../data/franchise/mlb_franchise.csv')

In [111]:
mlb_franchise

,year,team,league,total_games,wins,losses,ties,season_win_loss_%,pythagorean_win_loss_%,division_finish,division_game_back,runs_for,runs_against,season_total_attendance,average_batter_age,average_pitcher_age,number_of_batters,number_of_pitchers,top_player,manager
0,2019,HOU,AL West,162,107,55,0,0.660,0.660,1st of 5,--,920,640,2857367,28.9,29.9,45,26,A.Bregman (8.4),A.Hinch (107-55)
1,2018,HOU,AL West,162,103,59,0,0.636,0.675,1st of 5,--,797,534,2980549,28.1,30.0,41,22,A.Bregman (6.9),A.Hinch (103-59)
2,2017,HOU,AL West,162,101,61,0,0.623,0.611,1st of 5,--,896,700,2403671,28.8,28.5,46,27,J.Altuve (8.1),A.Hinch (101-61)
3,2016,HOU,AL West,162,84,78,0,0.519,0.515,3rd of 5,11,724,701,2306623,26.4,28.9,43,23,J.Altuve (7.7),A.Hinch (84-78)
4,2015,HOU,AL West,162,86,76,0,0.531,0.575,2nd of 5,2,729,618,2153585,26.3,29.4,46,24,D.Keuchel (6.7),A.Hinch (86-76)
5,2014,HOU,AL West,162,70,92,0,0.432,0.437,4th of 5,28,629,723,1751829,25.4,27.9,48,28,J.Altuve (6.1),B.Porter (59-79) and T.Lawless (11-13)
6,2013,HOU,AL West,162,51,111,0,0.315,0.354,5th of 5,45,610,848,1651883,25.9,27.2,50,26,J.Castro (4.1),B.Porter (51-111)
7,2012,HOU,NL Central,162,55,107,0,0.340,0.362,6th of 6,42,583,794,1607733,26.6,27.2,50,26,L.Harrell (3.1),B.Mills (39-82) and T.DeFrancesco (16-25)
8,2011,HOU,NL Central,162,56,106,0,0.346,0.384,6th of 6,40,615,796,2067016,28.2,27.2,47,23,C.Lee (3.9),B.Mills (56-106)
9,2010,HOU,NL Central,162,76,86,0,0.469,0.420,4th of 6,15,611,729,2331490,29.5,29.6,48,25,M.Bourn (5.5),B.Mills (76-86)


In [112]:
mlb_df = mlb_df.merge(mlb_franchise[['team', 'year', 'average_batter_age', 'average_pitcher_age', 'league']], on = ['team', 'year'])


In [114]:
mlb_df.to_csv('../data/clean_data/mlb_df.csv', index=False)